In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from functions import merge_data
from functions import load_medicare_data
from functions import load_respiratory_disease_data
from os.path import join as oj
import math
import re
import joblib
import load_data

In [3]:
county_info_keys = ['#Cases_3/22/2020', '#Deaths_3/22/2020', 'countyFIPS']
df = load_data.merge_hospital_and_county_data(hospital_info_keys='all', 
                                              county_info_keys=county_info_keys)


Index(['Unnamed: 0', 'COUNTYFIPS', 'COUNTY', 'STATE'], dtype='object')
loading county level data...


/accounts/projects/vision/chandan/covid-19-geographic-risk-prediction/functions/merge_data.py:40: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  diabetes = pd.read_csv(diabetes, skiprows = 2, skipfooter = 1)


ks Index(['countyFIPS', 'Respiratory Mortality'], dtype='object')


In [4]:
df.shape

(7143, 189)

In [ ]:
df['countyFIPS']

In [5]:
df.keys()

Index(['Unnamed: 0', 'CMS Certification Number', 'Facility Name_x',
       'Street Address_x', 'City_x', 'State_x', 'ZIP', 'System Affiliation',
       'Type of Facility', 'Total Employees',
       ...
       'Zip Code', 'Street Address 1', 'Street Address 2', 'City.1', 'State.1',
       'Zip Code.1', 'Facility ID_y', 'countyFIPS', '#Cases_3/22/2020',
       '#Deaths_3/22/2020'],
      dtype='object', length=189)

In [ ]:
df['#Cases_3/22/2020'].values[:500]

In [ ]:
df['countyFIPS'].values[:500]